# Deep Learning Overview

## Neural Network Components

- Input layer: takes raw features (e.g., image pixels as 2D/3D arrays).
- Hidden layers: weighted sums + biases pass through activations to learn representations.
- Activations: add nonlinearity (ReLU, sigmoid, tanh) so the model can fit complex patterns.
- Output layer: produces task-specific predictions (e.g., class scores for cat/dog/bird).
- Training: adjust weights/biases via backpropagation and gradient descent to reduce loss.

![Alt text](https://github.com/suthekshan/Agentic-Ai-Foundations/blob/main/images/nn.png?raw=1)

## Disadvantages of Feedforward Neural Networks

- **No memory** – cannot remember past inputs
- **No temporal dependency modeling** – treats each input independently
- **Fixed-size input only** – all features must be given at once
- **No sense of order or sequence**
- **Poor performance on sequential data** (text, speech, time series)

## Recurrent Neural Networks (RNNs)


## Why Do We Need Recurrent Neural Networks (RNNs)?

- **Have memory** – store information from previous time steps
- **Model temporal dependencies** in sequential data
- **Process variable-length sequences**
- **Order-aware** – understands sequence and context
- **Better suited for time-based data**

Recurrent Neural Networks (RNNs)

- Process sequences step by step, carrying a hidden state that summarizes prior tokens.
- Parameter sharing across time makes them data-efficient for sequential patterns.
- Limitations for text generation:
  - Vanishing/exploding gradients hinder learning long-range dependencies.
  - Hidden state is a bottleneck; context can fade over long spans.
  - Strictly sequential computation limits parallelism, slowing training/inference.

![Alt text](https://github.com/suthekshan/Agentic-Ai-Foundations/blob/main/images/rnn.png?raw=1)

## RNN - Text Generation

- Processes input **sequentially, one word at a time**
- Each word is converted into a **word embedding** \(c_t\)
- **Hidden state** \(h^{(t)}\) stores information from previous words
- Hidden state is updated using current input and past memory
- **Initial hidden state** \(h^{(0)}\) starts the sequence
- **Softmax output layer** produces probability distribution over vocabulary
- Predicts the **next word** based on previous context
- Captures **temporal dependencies** in language


<p align="center">
  <img src="https://github.com/suthekshan/Agentic-Ai-Foundations/blob/main/images/rnntextgeneration.png?raw=1" alt="Alt text">
</p>

## RNN Disadvantages

- **Recurrent computation is slow** due to sequential processing
- **Difficult to learn long-term dependencies**
- **Vanishing and exploding gradient problems**
- **Hard to access information from many time steps back**
- **Training is computationally expensive**

## Need for LSTM (Long Short-Term Memory)

- Overcomes **vanishing gradient problem** in RNNs
- Effectively learns **long-term dependencies**
- Retains important information and **forgets irrelevant data**
- Performs better on **long sequences**
- More stable and reliable training than standard RNNs
- Widely used in **language modeling, speech recognition, and time-series tasks**

## LSTM Architecture – Brief Overview

- LSTM is a special type of RNN designed to model **long-term dependencies**
- Consists of a **cell state (Cₜ)** that acts as long-term memory
- Uses **three gates** to control information flow:
  - **Forget Gate** – decides what information to remove from the cell state
  - **Input Gate** – decides what new information to store
  - **Output Gate** – decides what information to output
- Gates use **sigmoid activation** to control data flow
- **Tanh activation** is used to scale candidate values
- Cell state flows through the network with minimal modification, reducing information loss
- Produces a **hidden state (hₜ)** at each time step for output or next step processing

<p align="center">
  <img src="https://github.com/suthekshan/Agentic-Ai-Foundations/blob/main/images/lstm.png?raw=1" alt="Alt text">
</p>

## LSTM - Text Generation

<p align="center">
  <img src="https://github.com/suthekshan/Agentic-Ai-Foundations/blob/main/images/lstmtextgen.png?raw=1" alt="Alt text">
</p>


---

### 📦 1. Install Required Packages


In [7]:
!pip install --upgrade tensorflow numpy


### 2.Import necessary Libs

In [30]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 3. Dataset Preparation Function (Fixed & Clean)

In [10]:
tokenizer = Tokenizer()

In [31]:

def dataset_preparation(data):

    corpus = data.lower().split("\n")

    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            input_sequences.append(token_list[:i + 1])

    max_sequence_len = max(len(seq) for seq in input_sequences)

    input_sequences = np.array(
        pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
    )

    predictors = input_sequences[:, :-1]
    labels = input_sequences[:, -1]

    labels = to_categorical(labels, num_classes=total_words)

    return predictors, labels, max_sequence_len, total_words


### 4. Model Creation Function

In [32]:
def create_model(predictors, labels, max_sequence_len, total_words):

    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    earlystop = EarlyStopping(
        monitor='loss',
        patience=5,
        restore_best_weights=True
    )

    model.fit(
        predictors,
        labels,
        epochs=100,
        verbose=1,
        callbacks=[earlystop]
    )

    model.summary()
    return model


### ✍️ 5. Text Generation Function (Optimized)

In [33]:
def generate_text(seed_text, next_words, max_sequence_len, model):

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences(
            [token_list],
            maxlen=max_sequence_len - 1,
            padding='pre'
        )

        predicted = np.argmax(model.predict(token_list, verbose=0))
        output_word = tokenizer.index_word.get(predicted, "")

        seed_text += " " + output_word

    return seed_text


6. Load Data from Google Drive


In [35]:
from google.colab import drive
drive.mount('/content/drive')

data = open('/content/drive/MyDrive/Agentic-AIOT-Workshop/sample1.txt').read()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 7. Train & Generate Text

In [36]:

predictors, labels, max_sequence_len, total_words = dataset_preparation(data)

model = create_model(
    predictors,
    labels,
    max_sequence_len,
    total_words
)

print(generate_text("it can process", 5, max_sequence_len, model))
print("--Over--")


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 0.0181 - loss: 4.2060
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.0530 - loss: 4.1944
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.0608 - loss: 4.1684
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.0349 - loss: 4.1141
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0569 - loss: 4.1053
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.0530 - loss: 4.0238
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.0814 - loss: 3.9198
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.1099 - loss: 3.8218
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.1074 - loss: 3.7633
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.0775 - loss: 3.7237
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.0775 - loss: 3.6281
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step - accuracy: 

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 28, 100)        │         6,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 28, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 28, 100)        │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 67)             │         6,767 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,034,603 (3.95 MB)

 Trainable params: 344,867 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 689,736 (2.63 MB)

it can process lstm only single data such
--Over--


### Generate text

In [37]:
print(
    generate_text(
        seed_text="lstm is",
        next_words=10,
        max_sequence_len=max_sequence_len,
        model=model
    )
)


lstm is lstm lstm feedback connections such as unsegmented connected handwriting recognition


In [38]:
print(generate_text( seed_text="deep learning",
        next_words=5,max_sequence_len=max_sequence_len,model=model)
)

deep learning lstm lstm is to tasks


⚠️ Why Output is Repetitive (Expected)
*   trained on very small text





## Learning Resources

### 📘 Neural Networks (Foundations)

- Neural Networks Explained (YouTube)  
  https://youtu.be/alfdI7S6wCY?si=PCvbro1kgdfPPjzn

---

### 📘 Recurrent Neural Networks (RNN)

- Text Generation using RNN (Interactive Tutorial)  
  https://artinte.github.io/deep-learning/text_generate_rnn.html

---

### 📘 LSTM

- Understanding LSTM Networks (Colah’s Blog)  
  https://colah.github.io/posts/2015-08-Understanding-LSTMs/

- Gentle Introduction to LSTM  
  https://machinelearningmastery.com/gentle-introduction-long-short-term-memory-networks-experts/

---

### 📘 Deep Learning References

- Deep Learning Book – RNN Chapter  
  https://www.deeplearningbook.org/contents/rnn.html

- TensorFlow RNN Guide  
  https://www.tensorflow.org/guide/keras/rnn

---

### 📘 Practical Tutorials

- Text Generation with TensorFlow  
  https://www.tensorflow.org/tutorials/text/text_generation

- Keras LSTM API Documentation  
  https://keras.io/api/layers/recurrent_layers/lstm/

---

### 📘 Research Papers

- Original LSTM Paper (Hochreiter & Schmidhuber)  
  https://www.bioinf.jku.at/publications/older/2604.pdf

- Empirical Evaluation of Gated RNNs  
  https://arxiv.org/abs/1409.2329
